In [1]:
# Load packages
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import pickle
import time
import scipy as scp
import scipy.stats as scps
from scipy.optimize import differential_evolution
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load my own functions
import dnnregressor_train_eval_keras as dnnk
import make_data_wfpt as mdw
from kde_training_utilities import kde_load_data
import ddm_data_simulation as ddm_sim
import boundary_functions as bf

In [2]:
# Load Model
model_path = '/home/afengler/git_repos/nn_likelihoods/keras_models/dnnregressor_kde_ddm_weibull_06_05_19_14_07_16/model_0' 
ckpt_path = '/home/afengler/git_repos/nn_likelihoods/keras_models/dnnregressor_kde_ddm_weibull_06_05_19_14_07_16/ckpt_0_final'
model = keras.models.load_model(model_path)
model.load_weights(ckpt_path)

In [8]:
# Initializations -----
n_runs = 300
n_samples = 2500


# NOTE PARAMETERS: WEIBULL: [v, a, w, node, shape, scale]
param_bounds = [(-2.5, 2.5), (0.5, 4), (0.15, 0.85), (0, 5), (1.01, 49), (0.01, 9.9)]


my_optim_columns = ['v_sim', 'a_sim', 'w_sim', 'node_sim', 'shape_sim', 'scale_sim',
                    'v_mle', 'a_mle', 'w_mle', 'node_mle', 'shape_mle', 'scale_mle', 'n_samples']

# Get parameter names in correct ordering:
dat = pickle.load(open('/home/afengler/git_repos/nn_likelihoods/data_storage/kde/weibull/train_test_data/test_features.pickle' , 
                       'rb'))

parameter_names = list(dat.keys())[:-2] # :-1 to get rid of 'rt' and 'choice' here

# Make columns for optimizer result table
p_sim = []
p_mle = []

for parameter_name in parameter_names:
    p_sim.append(parameter_name + '_sim')
    p_mle.append(parameter_name + '_mle')
    
my_optim_columns = p_sim + p_mle + ['n_samples']

# Initialize the data frame in which to store optimizer results
optim_results = pd.DataFrame(np.zeros((n_runs, len(my_optim_columns))), columns = my_optim_columns)
optim_results.iloc[:, 2 * len(parameter_names)] = n_samples

# define boundary
boundary = bf.weibull_bnd
boundary_multiplicative = False

# Define the likelihood function
def log_p(params = [0, 1, 0.9], model = [], data = [], parameter_names = []):
    # Make feature array
    feature_array = np.zeros((data[0].shape[0], len(parameter_names) + 2))
    
    # Store parameters
    cnt = 0
    for i in range(0, len(parameter_names), 1):
        feature_array[:, i] = params[i]
        cnt += 1
    
    # Store rts and choices
    feature_array[:, cnt] = data[0].ravel() # rts
    feature_array[:, cnt + 1] = data[1].ravel() # choices
    
    # Get model predictions
    prediction = model.predict(feature_array)
    
    # Some post-processing of predictions
    prediction[prediction < 1e-29] = 1e-29
    
    return(- np.sum(np.log(prediction)))  

def make_params(param_bounds = []):
    params = np.zeros(len(param_bounds))
    
    for i in range(len(params)):
        params[i] = np.random.uniform(low = param_bounds[i][0], high = param_bounds[i][1])
        
    return
# ---------------------

In [ ]:
# Main loop ----------- TD: Parallelize
for i in range(0, n_runs, 1): 
    
    # Get start time
    start_time = time.time()
    
#     # Sample parameters
#     v_sim = np.random.uniform(high = v_range[1], low = v_range[0])
#     a_sim = np.random.uniform(high = a_range[1], low = a_range[0])
#     w_sim = np.random.uniform(high = w_range[1], low = w_range[0])

#     #c1_sim = np.random.uniform(high = c1_range[1], low = c1_range[0])
#     #c2_sim = np.random.uniform(high = c2_range[1], low = c2_range[0])
#     node_sim = np.random.uniform(high = node_range[1], low = node_range[0])
#     shape_sim = np.random.uniform(high = shape_range[1], low = shape_range[0])
#     scale_sim = np.random.uniform(high = scale_range[1], low = scale_range[0])

    tmp_params = make_params(param_bounds = param_bounds)
    
    # Store in output file
    optim_results.iloc[i, :len(parameter_names)] = tmp_params
    
    # Print some info on run
    print('Parameters for run ' + str(i) + ': ')
    print([v_sim, a_sim, w_sim, node_sim, shape_sim, scale_sim])
    
    # Run model simulations
    ddm_dat_tmp = ddm_sim.ddm_flexbound_simulate(v = tmp_params[0],
                                                 a = tmp_params[1],
                                                 w = tmp_params[2],
                                                 s = 1,
                                                 delta_t = 0.001,
                                                 max_t = 20,
                                                 n_samples = n_samples,
                                                 boundary_fun = boundary, # function of t (and potentially other parameters) that takes in (t, *args)
                                                 boundary_multiplicative = boundary_multiplicative, # CAREFUL: CHECK IF BOUND
                                                 boundary_params = {'node': tmp_params[3], 
                                                                    'shape': tmp_params[4],
                                                                    'scale': tmp_params[5]})
        
    # Print some info on run
    print('Mean rt for current run: ')
    print(np.mean(ddm_dat_tmp[0]))
    
    # Run optimizer
    out = differential_evolution(log_p, 
                                 bounds = [(v_range[0],v_range[1]), 
                                           (a_range[0], a_range[1]), 
                                           (w_range[0], w_range[1]),
                                           (node_range[0], node_range[1]),
                                           (shape_range[0], shape_range[1]),
                                           (scale_range[0], scale_range[1])], 
                                 args = (model, ddm_dat_tmp, parameter_names), 
                                 popsize = 30,
                                 disp = True)
    
    # Print some info
    print('Solution vector of current run: ')
    print(out.x)
    
    print('The run took: ')
    elapsed_time = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
    
    # Store result in output file
    optim_results.iloc[i, len(parameter_names):(2*len(parameter_names))] = out.x
# -----------------------

# Save optimization results to file
optim_results.to_csv(os.getcwd() + '/experiments/kde_ddm_weibull_mle/optim_results.csv')

Parameters for run 0: 
[-0.7830763824720766, 1.5490168447634234, 0.19819589306034507, 0.48161376192033867, 6.756848157144321, 5.01930509634567]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.7186211999999851
differential_evolution step 1: f(x)= 1732.9
differential_evolution step 2: f(x)= 1717.6
differential_evolution step 3: f(x)= 1637.73
differential_evolution step 4: f(x)= 1637.73
differential_evolution step 5: f(x)= 1621.76
differential_evolution step 6: f(x)= 1621.76
differential_evolution step 7: f(x)= 1607.92
differential_evolution step 8: f(x)= 1599.18
differential_evolution step 9: f(x)= 1485.39
differential_evolution step 10: f(x)= 1485.39
differential_evolution step 11: f(x)= 1485.39
differential_evolution step 12: f(x)= 1485.39
differential_evolution step 13: f(x)= 1442.56
differential_evolution step 14: f(x)= 1442.56
differential_evolution step 15: f(x)= 1442.56
differential_evolution step 16: f(x)= 1442.56
differential_e

differential_evolution step 23: f(x)= -1756.3
differential_evolution step 24: f(x)= -1756.3
differential_evolution step 25: f(x)= -1756.3
differential_evolution step 26: f(x)= -1850.62
differential_evolution step 27: f(x)= -1850.62
differential_evolution step 28: f(x)= -1923.64
differential_evolution step 29: f(x)= -1923.64
differential_evolution step 30: f(x)= -1923.64
differential_evolution step 31: f(x)= -1923.64
differential_evolution step 32: f(x)= -1923.64
differential_evolution step 33: f(x)= -1923.64
differential_evolution step 34: f(x)= -2185.28
differential_evolution step 35: f(x)= -2185.28
differential_evolution step 36: f(x)= -2185.28
differential_evolution step 37: f(x)= -2185.28
differential_evolution step 38: f(x)= -2185.28
differential_evolution step 39: f(x)= -2185.28
differential_evolution step 40: f(x)= -2185.28
differential_evolution step 41: f(x)= -2185.28
differential_evolution step 42: f(x)= -2185.28
differential_evolution step 43: f(x)= -2210.2
differential_evol

differential_evolution step 5: f(x)= 5539.94
differential_evolution step 6: f(x)= 5539.94
differential_evolution step 7: f(x)= 5539.94
differential_evolution step 8: f(x)= 5539.94
differential_evolution step 9: f(x)= 5276.33
differential_evolution step 10: f(x)= 5276.33
differential_evolution step 11: f(x)= 5276.33
differential_evolution step 12: f(x)= 5276.33
differential_evolution step 13: f(x)= 5246.02
differential_evolution step 14: f(x)= 5246.02
differential_evolution step 15: f(x)= 5246.02
differential_evolution step 16: f(x)= 5246.02
differential_evolution step 17: f(x)= 5215.33
differential_evolution step 18: f(x)= 5215.33
differential_evolution step 19: f(x)= 5215.33
differential_evolution step 20: f(x)= 5215.33
differential_evolution step 21: f(x)= 5213.47
differential_evolution step 22: f(x)= 5213.47
differential_evolution step 23: f(x)= 5213.47
differential_evolution step 24: f(x)= 5192.03
differential_evolution step 25: f(x)= 5179.86
differential_evolution step 26: f(x)= 5

differential_evolution step 19: f(x)= 6313.78
differential_evolution step 20: f(x)= 6313.78
differential_evolution step 21: f(x)= 6313.78
differential_evolution step 22: f(x)= 6313.78
differential_evolution step 23: f(x)= 6313.78
differential_evolution step 24: f(x)= 6313.78
differential_evolution step 25: f(x)= 6313.78
differential_evolution step 26: f(x)= 6313.78
differential_evolution step 27: f(x)= 6313.78
differential_evolution step 28: f(x)= 6313.78
differential_evolution step 29: f(x)= 6313.78
differential_evolution step 30: f(x)= 6313.78
Solution vector of current run: 
[ 0.46895685  2.4385509   0.5743943   4.55674727 48.26313149  3.99761062]
The run took: 
00:07:08
Parameters for run 9: 
[-1.1313853591120977, 3.3642806424246063, 0.4378170405920663, 0.010048813190230987, 12.579603245471427, 7.47555053347391]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
2.6337307999999005
differential_evolution step 1: f(x)= 3868.06
differenti

differential_evolution step 89: f(x)= 572.086
differential_evolution step 90: f(x)= 572.086
differential_evolution step 91: f(x)= 572.086
differential_evolution step 92: f(x)= 570.215
differential_evolution step 93: f(x)= 570.215
differential_evolution step 94: f(x)= 570.215
differential_evolution step 95: f(x)= 570.215
differential_evolution step 96: f(x)= 570.215
Solution vector of current run: 
[-2.23360401  2.689136    0.40255245  0.          1.11125081  0.11115329]
The run took: 
00:12:22
Parameters for run 11: 
[0.8036506318681691, 3.0895432719830143, 0.8235703748561655, 4.9603481899049005, 45.27265242964932, 2.055496867925082]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
1.359040799999969
differential_evolution step 1: f(x)= 2896.91
differential_evolution step 2: f(x)= 2888.08
differential_evolution step 3: f(x)= 2861.25
differential_evolution step 4: f(x)= 2697.72
differential_evolution step 5: f(x)= 2697.72
differential_evol

...

differential_evolution step 94: f(x)= -568.73
differential_evolution step 95: f(x)= -569.752
differential_evolution step 96: f(x)= -569.911
differential_evolution step 97: f(x)= -569.911
differential_evolution step 98: f(x)= -570.054
differential_evolution step 99: f(x)= -570.054
differential_evolution step 100: f(x)= -570.054
differential_evolution step 101: f(x)= -570.381
differential_evolution step 102: f(x)= -570.781
differential_evolution step 103: f(x)= -570.781
Solution vector of current run: 
[-2.23019597e+00  5.00150202e-01  5.37504722e-01  9.06442974e-04
  1.11027278e+00  1.16019238e-01]
The run took: 
00:13:32
Parameters for run 258: 
[1.9873506097521805, 2.602315978668052, 0.18830015427486665, 3.528237883710617, 30.7089268212218, 1.671088531860045]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
2.076486799999887
differential_evolution step 1: f(x)= 2987.52
differential_evolution step 2: f(x)= 2987.52
differential_evolution 

differential_evolution step 29: f(x)= 1764.11
differential_evolution step 30: f(x)= 1764.11
differential_evolution step 31: f(x)= 1764.11
differential_evolution step 32: f(x)= 1764.11
differential_evolution step 33: f(x)= 1732.84
differential_evolution step 34: f(x)= 1732.84
differential_evolution step 35: f(x)= 1691.13
differential_evolution step 36: f(x)= 1691.13
differential_evolution step 37: f(x)= 1691.13
differential_evolution step 38: f(x)= 1691.13
differential_evolution step 39: f(x)= 1691.13
differential_evolution step 40: f(x)= 1691.13
differential_evolution step 41: f(x)= 1691.13
differential_evolution step 42: f(x)= 1691.13
differential_evolution step 43: f(x)= 1691.13
differential_evolution step 44: f(x)= 1691.13
differential_evolution step 45: f(x)= 1688.83
differential_evolution step 46: f(x)= 1673.11
differential_evolution step 47: f(x)= 1673.11
differential_evolution step 48: f(x)= 1673.11
differential_evolution step 49: f(x)= 1629.22
differential_evolution step 50: f(

differential_evolution step 49: f(x)= 2643.44
differential_evolution step 50: f(x)= 2643.44
differential_evolution step 51: f(x)= 2633.64
differential_evolution step 52: f(x)= 2631.6
differential_evolution step 53: f(x)= 2631.6
differential_evolution step 54: f(x)= 2631.6
differential_evolution step 55: f(x)= 2631.6
differential_evolution step 56: f(x)= 2631.6
differential_evolution step 57: f(x)= 2631.6
differential_evolution step 58: f(x)= 2631.6
differential_evolution step 59: f(x)= 2631.6
differential_evolution step 60: f(x)= 2631.6
Solution vector of current run: 
[-1.54806321  3.99720238  0.40648994  0.00835863  1.23479936  0.14159113]
The run took: 
00:09:59
Parameters for run 263: 
[2.4820466893990583, 1.2143090278012294, 0.4213653287604049, 4.405974602573545, 33.210961251023576, 1.8200515504298365]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.5768015999999979
differential_evolution step 1: f(x)= 235.531
differential_evolut

differential_evolution step 55: f(x)= 1866.69
differential_evolution step 56: f(x)= 1866.69
differential_evolution step 57: f(x)= 1866.69
Solution vector of current run: 
[-1.19170605  0.70524387  0.77644552  4.44454609 24.72659766  9.20502857]
The run took: 
00:07:44
Parameters for run 265: 
[0.3207565171868372, 0.7887745040847169, 0.7155612350975538, 0.6721240434334164, 18.7455292781451, 8.446160424986916]
0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
0.4765839999999931
differential_evolution step 1: f(x)= 1834.51
differential_evolution step 2: f(x)= 1621.3
differential_evolution step 3: f(x)= 1512.22
differential_evolution step 4: f(x)= 1512.22
differential_evolution step 5: f(x)= 1421.79
differential_evolution step 6: f(x)= 1392.84
differential_evolution step 7: f(x)= 1392.84
differential_evolution step 8: f(x)= 1335.05
differential_evolution step 9: f(x)= 1299.88
differential_evolution step 10: f(x)= 1279.73
differential_evolutio

0  datapoints sampled
1000  datapoints sampled
2000  datapoints sampled
Mean rt for current run: 
2.510424799999958
differential_evolution step 1: f(x)= 6993.64
differential_evolution step 2: f(x)= 6359.67
differential_evolution step 3: f(x)= 6359.67
differential_evolution step 4: f(x)= 6308.37
differential_evolution step 5: f(x)= 6300.47
differential_evolution step 6: f(x)= 6300.47
differential_evolution step 7: f(x)= 6289.2
differential_evolution step 8: f(x)= 6270.35
differential_evolution step 9: f(x)= 6252.01
differential_evolution step 10: f(x)= 6252.01
differential_evolution step 11: f(x)= 6241.36
differential_evolution step 12: f(x)= 6241.36
differential_evolution step 13: f(x)= 6241.36
differential_evolution step 14: f(x)= 6241.36
differential_evolution step 15: f(x)= 6241.16
Solution vector of current run: 
[ 0.25304158  1.39105759  0.47049989  4.83559549 41.19368762  1.34233813]
The run took: 
00:04:04
Parameters for run 268: 
[-2.3935799805621483, 2.033177856546146, 0.81680

In [ ]:
# Read in results
optim_results = pd.read_csv(os.getcwd() + '/experiments/ddm_flexbound_kde_mle_fix_v_0_c1_0_w_unbiased_arange_2_3/optim_results.csv')

In [ ]:
plt.scatter(optim_results['v_sim'], optim_results['v_mle'], c = optim_results['c2_mle'])

In [ ]:
# Regression for v
reg = LinearRegression().fit(np.expand_dims(optim_results['v_mle'], 1), np.expand_dims(optim_results['v_sim'], 1))
reg.score(np.expand_dims(optim_results['v_mle'], 1), np.expand_dims(optim_results['v_sim'], 1))

In [ ]:
plt.scatter(optim_results['a_sim'], optim_results['a_mle'], c = optim_results['c2_mle'])

In [ ]:
# Regression for a
reg = LinearRegression().fit(np.expand_dims(optim_results['a_mle'], 1), np.expand_dims(optim_results['a_sim'], 1))
reg.score(np.expand_dims(optim_results['a_mle'], 1), np.expand_dims(optim_results['a_sim'], 1))

In [ ]:
plt.scatter(optim_results['w_sim'], optim_results['w_mle'])

In [ ]:
# Regression for w
reg = LinearRegression().fit(np.expand_dims(optim_results['w_mle'], 1), np.expand_dims(optim_results['w_sim'], 1))
reg.score(np.expand_dims(optim_results['w_mle'], 1), np.expand_dims(optim_results['w_sim'], 1))

In [ ]:
plt.scatter(optim_results['c1_sim'], optim_results['c1_mle'])

In [ ]:
# Regression for c1
reg = LinearRegression().fit(np.expand_dims(optim_results['c1_mle'], 1), np.expand_dims(optim_results['c1_sim'], 1))
reg.score(np.expand_dims(optim_results['c1_mle'], 1), np.expand_dims(optim_results['c1_sim'], 1))

In [ ]:
plt.scatter(optim_results['c2_sim'], optim_results['c2_mle'], c = optim_results['a_mle'])

In [ ]:
# Regression for w
reg = LinearRegression().fit(np.expand_dims(optim_results['c2_mle'], 1), np.expand_dims(optim_results['c2_sim'], 1))
reg.score(np.expand_dims(optim_results['c2_mle'], 1), np.expand_dims(optim_results['c2_sim'], 1))